In [0]:
from pyspark.sql.types import StructType, StructField, StringType, LongType
from pyspark.sql.functions import count, col

# # Catalog and table name
catalog_name = "datricks_airflow_azure"
schema_name = "gold"
table_name = "project_brewery"
full_table_name = f"{catalog_name}.{schema_name}.{table_name}"


# Apply comments
def adicionaComentariosTabela(catalog, schema, table, table_comment, col_comments):
    spark.sql(f"COMMENT ON TABLE {catalog}.{schema}.{table} IS '{table_comment}'")
    schema_struct = spark.table(f"{catalog}.{schema}.{table}").schema
    for field in schema_struct:
        if field.name in col_comments:
            tipo = field.dataType.simpleString()
            comentario = col_comments[field.name]
            sql = f"ALTER TABLE {catalog}.{schema}.{table} CHANGE COLUMN {field.name} {field.name} {tipo} COMMENT '{comentario}'"
            spark.sql(sql)

# Define the schema
schema = StructType([
    StructField("state_province", StringType(), True),
    StructField("brewery_type", StringType(), True),
    StructField("brewery_count", LongType(), True)
])


table_comment = "Aggregated brewery count per type and location from Silver layer"


column_comments = {
    "state_province": "Province or state where the brewery is located",
    "brewery_type": "Type/category of brewery",
    "brewery_count": "Number of breweries in the given location and type"
}

# Check table exists
def table_exists(catalog, schema, table):
    try:
        spark.table(f"{catalog}.{schema}.{table}")
        return True
    except:
        return False

# Read Sillver Data
df_silver = spark.read.table("datricks_airflow_azure.silver.project_brewery")

# Agregação
df_gold = df_silver.groupBy("state_province", "brewery_type").agg(
    count("*").alias("brewery_count")
)

# Cast columns dynamically based on the schema
for field in schema.fields:
    df_gold = df_gold.withColumn(field.name, col(field.name).cast(field.dataType))

# Write Table
if table_exists(catalog_name, schema_name, table_name):
    df_gold.write \
        .format("delta") \
        .mode("overwrite") \
        .partitionBy("state_province") \
        .saveAsTable(name = f"{catalog_name}.{schema_name}.{table_name}")
    print(f"Overwrite to existing Delta table: {full_table_name}")

else:
    # Create schema if not exists
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_name}")

    # If not exists, write and register in catalog
    df_gold.write \
        .mode("overwrite") \
        .partitionBy("state_province") \
        .option("overwriteSchema", "true") \
        .saveAsTable(name = f"{catalog_name}.{schema_name}.{table_name}")

    adicionaComentariosTabela(catalog_name, schema_name, table_name, table_comment, column_comments)
    print(f"Delta table created and registered: {full_table_name}")